ADO (Automated Data Operations) classification graph for "bad exposure" and "credential expired" 

In [1]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

# Define your custom state for classification
class IssueState(TypedDict):
    issue: str
    classification: Literal["bad_exposure", "credential_expired", "unknown"]



In [2]:
def classify_issue(state: IssueState) -> IssueState:
    issue = state["issue"].lower()
    if "exposure" in issue:
        state["classification"] = "bad_exposure"
    elif "credential" in issue or "expired" in issue:
        state["classification"] = "credential_expired"
    else:
        state["classification"] = "unknown"
    return state


In [4]:
def handle_bad_exposure(state: IssueState) -> IssueState:
    print(" Handling bad exposure issue.")
    return state

def handle_credential_expired(state: IssueState) -> IssueState:
    print(" Handling credential expired issue.")
    return state

def handle_unknown(state: IssueState) -> IssueState:
    print(" Unknown issue. Needs manual review.")
    return state


In [5]:
def route_classification(state: IssueState) -> str:
    return state["classification"]


In [6]:
# Step-by-step build
builder = StateGraph(IssueState)

builder.add_node("classify_issue", classify_issue)
builder.add_node("handle_bad_exposure", handle_bad_exposure)
builder.add_node("handle_credential_expired", handle_credential_expired)
builder.add_node("handle_unknown", handle_unknown)

# Start → classify
builder.add_edge("classify_issue", END)

# Add conditional routing from classify_issue
builder.add_conditional_edges(
    "classify_issue",
    route_classification,
    {
        "bad_exposure": "handle_bad_exposure",
        "credential_expired": "handle_credential_expired",
        "unknown": "handle_unknown"
    }
)

# Connect all handlers to END
builder.add_edge("handle_bad_exposure", END)
builder.add_edge("handle_credential_expired", END)
builder.add_edge("handle_unknown", END)

# Add entry point
builder.set_entry_point("classify_issue")

# Compile
graph = builder.compile()


In [12]:
# Run with a sample input for bad exposure
output = graph.invoke({"issue": "The exposure settings are incorrect."})
print("Output:", output)


 Handling bad exposure issue.
Output: {'issue': 'The exposure settings are incorrect.', 'classification': 'bad_exposure'}


In [13]:
# Run with a sample input for credential expired
output = graph.invoke({"issue": "The credentials have expired for this system."})
print("Output:", output)



 Handling credential expired issue.
Output: {'issue': 'The credentials have expired for this system.', 'classification': 'credential_expired'}


In [14]:
# Run with an unknown issue
output = graph.invoke({"issue": "System is overheating."})
print("Output:", output)


 Unknown issue. Needs manual review.
Output: {'issue': 'System is overheating.', 'classification': 'unknown'}
